# Appending TD with `user_prompt` + Chain of Thought

In [ ]:
%pip install langchain-openai langchain-core langchain-community 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter OpenAI API Key: ")

In [2]:
from langchain_openai import ChatOpenAI

openai_model = "gpt-4.1"
# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model)

In [3]:
user_prompt = """
    Desired Workflow: {desired_workflow}
    List of Devices as WoT Thing Descriptions: \n {thing_directory}
    """

In [4]:
import sys
import os

parent_dir = os.path.abspath("..")
sys.path.append(parent_dir)

import utils


In [5]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(utils.system_prompt_cot),
    HumanMessagePromptTemplate.from_template(user_prompt)
])

In [6]:
prompt_template.input_variables

['desired_workflow', 'thing_directory']

In [7]:
# for testing purposes
print(prompt_template.format(desired_workflow="ipsum dolor", thing_directory="LOREM IPSUM"))

System: You are an expert IoT system developer, proficient in Web of Things (WoT) descriptions and Node-RED workflow programming.

Your task:
- Take the user's IoT system request (written in natural language) and the list of available WoT Thing Descriptions.
- Internally break the problem down into subproblems.
- Internally solve each subproblem step-by-step.
- Internally merge the solutions into a full Node-RED workflow.
- Ensure all node IDs are unique.
- Ensure all quote marks are escaped properly.
- Ensure the final output is valid JSON representing the Node-RED workflow.

Important:
- You MUST NOT output your reasoning, analysis, chain of thought, or subproblem list.
- You MUST ONLY output the final JSON workflow, with no extra text.

Human: 
    Desired Workflow: ipsum dolor
    List of Devices as WoT Thing Descriptions: 
 LOREM IPSUM
    


In [8]:
pipeline = (
    {
        "thing_directory": lambda x: x["thing_directory"],
        "desired_workflow": lambda x: x["desired_workflow"]
    }
    | prompt_template
    | llm
    | {"generated_workflow": lambda x: x.content}
)

In [9]:
response = pipeline.invoke({
    "thing_directory": utils.smart_home_td_full,
    "desired_workflow": utils.smart_home_system_description
})

In [10]:
response

{'generated_workflow': '[\n    {\n        "id": "n1a1b1c1d1e1f1g1",\n        "type": "websocket in",\n        "z": "main",\n        "name": "WashingMachine finishedCycle",\n        "server": "",\n        "client": "",\n        "path": "ws://localhost:1880/things/wm/finishedCycle",\n        "x": 120,\n        "y": 80,\n        "wires": [\n            [\n                "n2a2b2c2d2e2f2g2"\n            ]\n        ]\n    },\n    {\n        "id": "n2a2b2c2d2e2f2g2",\n        "type": "http request",\n        "z": "main",\n        "name": "Blink LEDs",\n        "method": "POST",\n        "ret": "txt",\n        "url": "http://localhost:1880/things/leds/blink",\n        "headers": {},\n        "x": 350,\n        "y": 80,\n        "wires": [\n            []\n        ]\n    },\n    {\n        "id": "n3a3b3c3d3e3f3g3",\n        "type": "websocket in",\n        "z": "main",\n        "name": "MotionSensor motionDetected",\n        "server": "",\n        "client": "",\n        "path": "ws://localhost

In [11]:
# formatting the response

import json

# Parse twice: first to extract string, second to get proper JSON
workflow_str = response['generated_workflow']
workflow = json.loads(workflow_str)

# Now `workflow` is a list of dicts — ready for Node-RED
print(json.dumps(workflow, indent=2))

[
  {
    "id": "n1a1b1c1d1e1f1g1",
    "type": "websocket in",
    "z": "main",
    "name": "WashingMachine finishedCycle",
    "server": "",
    "client": "",
    "path": "ws://localhost:1880/things/wm/finishedCycle",
    "x": 120,
    "y": 80,
    "wires": [
      [
        "n2a2b2c2d2e2f2g2"
      ]
    ]
  },
  {
    "id": "n2a2b2c2d2e2f2g2",
    "type": "http request",
    "z": "main",
    "name": "Blink LEDs",
    "method": "POST",
    "ret": "txt",
    "url": "http://localhost:1880/things/leds/blink",
    "headers": {},
    "x": 350,
    "y": 80,
    "wires": [
      []
    ]
  },
  {
    "id": "n3a3b3c3d3e3f3g3",
    "type": "websocket in",
    "z": "main",
    "name": "MotionSensor motionDetected",
    "server": "",
    "client": "",
    "path": "ws://localhost:1880/things/motionsensor/motiondetected",
    "x": 120,
    "y": 180,
    "wires": [
      [
        "n4a4b4c4d4e4f4g4"
      ]
    ]
  },
  {
    "id": "n4a4b4c4d4e4f4g4",
    "type": "http request",
    "z": "main",
